In [ ]:
import pandas as pd
from fastai.vision.all import *
from pathlib import Path
import itertools
import progressbar

In [ ]:
DANBOORU_PATH = Path('/home/lleonard/Documents/datasets/danbooru/0/danbooru2020/')
EXTENSIONS = ['png', 'jpg']

In [ ]:
i = 0
def file_exists(images_id):
    with progressbar.ProgressBar(max_value=len(images_id)) as bar:
        def single_image_exists(image_id):
            global i
            last_4_digit = str(image_id % 1000).zfill(4)
            path_to_glob = str(DANBOORU_PATH / '512px' / last_4_digit / str(image_id)) + '.'
            globbed = list(itertools.chain(*[glob.glob(path_to_glob + ext) for ext in EXTENSIONS]))
            bar.update(i)
            i = i + 1
            if len(globbed) == 0:
                return False
            return True

        return images_id.apply(single_image_exists)
    
    

ratings = pd.read_csv('/home/lleonard/Documents/datasets/danbooru/0/danbooru2020/ratings_tags_0000.csv')
print(ratings)
ratings = ratings[ratings['id'] % 1000 < 50]
ratings = ratings[file_exists(ratings['id'])]
ratings = ratings[['id', 'rating', 'tags']]

In [ ]:
ratings = pd.read_csv('clean_0000.csv')

In [ ]:
ratings.groupby('rating').count()

In [ ]:
sampled_ratings = ratings[ratings['rating'] != 's'].append(ratings[ratings['rating'] == 's'].sample(15000))
sampled_ratings.groupby('rating').count()

In [ ]:
# ratings.to_csv('clean_0000.csv', index=False)

In [ ]:
def get_x(row):
    image_id = str(row['id'])
    mod_1000 = str(row['id'] % 1000)
    path_to_glob = str(DANBOORU_PATH / '512px' / mod_1000.zfill(4) / str(image_id)) + '.'
    return list(itertools.chain(*[glob.glob(path_to_glob + ext) for ext in EXTENSIONS]))[0]
    

def get_y(row):
    return row['rating']

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   item_tfms=Resize((224,224)),
                   batch_tfms=[*aug_transforms(),Normalize()],
                   get_x = get_x, get_y = get_y)
dblock.summary(sampled_ratings)


In [ ]:
dataloaders = dblock.dataloaders(sampled_ratings, bs=128)
dataloaders.show_batch()

In [ ]:
learner = cnn_learner(dataloaders, models.vgg16_bn, metrics = accuracy).to_fp16()
learner.model

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(100, 3e-3)

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit_one_cycle(100, slice(3e-5, 3e-3))

In [ ]:
learner.save('danbooru_vgg_classifier')

In [ ]:
torch.cuda.empty_cache()
dataloaders = dblock.dataloaders(ratings, bs=64)
model = model.eval().to('cuda')
data = iter(dataloaders.valid)

In [ ]:
import torchvision

item = next(data)
grid = torchvision.utils.make_grid(item[0])
show_image(grid)
preds = model(item[0])
print(preds[preds[:, 0].argmax(), :])
show_image(item[0][preds[:, 0].argmax()])
del preds
del item

In [ ]:
import fastai
fastai.__version__, torch.__version__